# Model builder
## Life Cycle Assessment of ``{Operational Energy Moerschberg for 1 year: 1}``
###[CONTEXT]
The model relies on datapackages to ensure replicability of the calculation.


In [13]:
import bw2data as bd
import bw2calc as bc
import bw2io as bi
import bw_processing as bwp
import numpy as np
from pathlib import Path
import pandas as pd
bd.projects.set_current('moeschpond1')


In [18]:
# bd.projects.migrate_project_25()

Updating all LCIA methods
Title: Brightway2 LCIA methods objects:
  Started: 10/27/2022 19:37:03
  Finished: 10/27/2022 19:37:55
  Total time elapsed: 00:00:52
  CPU %: 99.90
  Memory %: 0.25
Updating all LCI databases
Title: Brightway2 LCI databases objects:
  Started: 10/27/2022 19:37:55
  Finished: 10/27/2022 19:38:52
  Total time elapsed: 00:00:56
  CPU %: 100.00
  Memory %: 0.27


In [43]:
bd.databases

Databases dictionary with 11 object(s):
	EI 3.8 cutoff image SSP2-Base 2050
	EI 3.8 cutoff image SSP2-RCP19 2050
	EI 3.8 cutoff image SSP2-RCP26 2050
	EI 3.8 cutoff remind SSP2-Base 2050
	EI 3.8 cutoff remind SSP2-PkBudg1150 2050
	EI 3.8 cutoff remind SSP2-PkBudg500 2050
	biosphere3
	ei 3.8 cutoff
	energy_moeschberg
	super_db_2022-10-26
	super_db_2022-10-27

In [101]:
# bg_base = "EI 3.8 cutoff image SSP2-Base 2050"
# bg_19 = "EI 3.8 cutoff image SSP2-RCP19 2050"

sce_names_dict={"id_ei38":"ei 3.8 cutoff",
"id_pre_base":"EI 3.8 cutoff image SSP2-Base 2050",
 "id_pre_19":"EI 3.8 cutoff image SSP2-RCP19 2050",
 "id_pre_26":"EI 3.8 cutoff image SSP2-RCP26 2050",
"id_re_base":"EI 3.8 cutoff remind SSP2-Base 2050",
"id_re_1150":"EI 3.8 cutoff remind SSP2-PkBudg1150 2050",
"id_re_1150":"EI 3.8 cutoff remind SSP2-PkBudg500 2050"
}

# bgs = [bg_base, bg_19]

In [106]:
imp = bi.ExcelImporter('./data/inputs/SIB inventory_v2_working.xlsx')
imp.apply_strategies()
imp.match_database("ei 3.8 cutoff", fields=('name','unit','location', 'reference product'))
#imp.match_database("ecoinvent_remind_SSP2-Base_2020", fields=('name','unit','location','reference product'))
imp.match_database(fields=('name', 'unit', 'location'))
imp.statistics()
imp.write_excel(only_unlinked=True)
list(imp.unlinked)

imp.write_database()


Extracted 1 worksheets in 0.02 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 7.04 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
14 datasets
140 exchanges
0 unlinked exchanges
  
Wrote matching file to:
/home/jupyter

In [ ]:
imp = ExcelImporter(r'C:\Users\treyer_k\Documents\GitHub\moeschpond\data\inputs\lci_solarpv_huimin_masterthesis_combined.xlsx')
imp.apply_strategies()
imp.match_database("ei 3.8 cutoff", fields=('name','unit','location'))
#imp.match_database("ecoinvent_remind_SSP2-Base_2020", fields=('name','unit','location','reference product'))
imp.match_database(fields=('name', 'unit', 'location'))
imp.statistics()
imp.write_excel(only_unlinked=True)
list(imp.unlinked)

imp.write_database()


In [45]:
# if "foreground" in bd.databases:
#     del bd.databases["foreground"]
    
foreground_importer = bi.ExcelImporter("./data/inputs/lci_moeschberg.xlsx")
foreground_importer.apply_strategies()
foreground_importer.match_database("biosphere3", fields=("name", "unit", "categories"))
foreground_importer.match_database("ei 3.8 cutoff", fields=("name", "unit", "location"))
foreground_importer.statistics()
foreground_importer.write_database()

In [46]:
fu = {bd.get_activity(database="energy_moeschberg", name = "energy demand, operational, Hotel Moeschberg"):1}

In [47]:
bd.get_activity(id=18352)

'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted' (kilowatt hour, CH, None)

In [48]:
### it returns functional_unit, data_objects, and a mapping dictionary
fu = {bd.get_activity(database="energy_moeschberg", name = "energy demand, operational, Hotel Moeschberg"):1}
fu, objects, remapping_dicts = bd.prepare_lca_inputs(fu, method=('IPCC 2013', 'climate change', 'GWP 100a'))

In [49]:
for obj in objects:
    print(obj.metadata["name"])

biosphere3
ei_3.8_cutoff
energy_moeschberg
ipcc-2013cg.bd5af3f67229a1cc291b8ecb7f316fcf.zip


In [50]:
data, _ = objects[2].get_resource("energy_moeschberg_technosphere_matrix.data")
indices, _ = objects[2].get_resource("energy_moeschberg_technosphere_matrix.indices")
flip, _ = objects[2].get_resource("energy_moeschberg_technosphere_matrix.flip")
unique_indices = set([a for b in indices for a in b])
mapping_act=dict.fromkeys(int(i) for i in unique_indices) #json accepts only int not int32
for i in unique_indices:
    mapping_act[i]=bd.get_activity(i)['name']
rows = [tup[0] for tup in indices]
cols = [tup[1] for tup in indices]

In [51]:
df = pd.DataFrame(columns=['id','name','location','reference product'],index=np.arange(len(unique_indices)))
n=0
for i in unique_indices: 
    df['id'][n]=(i)
    df['name'][n]=(bd.get_activity(i)['name'])
    df['location'][n]=(bd.get_activity(i)['location'])
    df['reference product'][n]=(bd.get_activity(i)['reference product'])
    n += 1

In [52]:
df

,id,name,location,reference product
0,6599,"electricity production, hydro, run-of-river",CH,"electricity, high voltage"
1,8302,"operation, solar collector system, Cu flat pla...",CH,"heat, central or small-scale, other than natur..."
2,10319,"electricity production, wind, 1-3MW turbine, o...",CH,"electricity, high voltage"
3,18352,"electricity production, photovoltaic, 3kWp sla...",CH,"electricity, low voltage"
4,20079,"heat and power co-generation, wood chips, 6667...",CH,"electricity, high voltage"
5,23758,"heat production, wood pellet, at furnace 25kW,...",CH,"heat, central or small-scale, other than natur..."
6,213909,"electricity supply, Hotel Moeschberg, 2021",CH,"electricity, low voltage"
7,213910,"electricity supply, Energie Grosshoechstetten ...",CH,"electricity, low voltage"
8,213911,"electricity, subsidised, 2021",CH,"electricity, low voltage"
9,213912,"heat supply, Hotel Moeschberg, 2021",CH,"heat, Hotel Moeschberg"


In [53]:
bd.get_activity(database = bg_19,
                name="electricity production, hydro, run-of-river",
                location="CH", 
                product = "electricity, high voltage").id

140945

In [ ]:
import pandas as pd

#initialize a dataframe
df = pd.DataFrame(
	[[21, 'Amol', 72, 67],
	[23, 'Lini', 78, 69],
	[32, 'Kiku', 74, 56],
	[52, 'Ajit', 54, 76]],
	columns=['rollno', 'name', 'physics', 'botony'])

print('DataFrame with default index\n', df)

#set multiple columns as index
df = df.set_index(['rollno','name'])

print('\nDataFrame with MultiIndex\n',df)

In [102]:
import pandas as pd 
id_all = []

sce_names_dict={"id_ei38":"ei 3.8 cutoff",
"id_pre_base":"EI 3.8 cutoff image SSP2-Base 2050",
 "id_pre_19":"EI 3.8 cutoff image SSP2-RCP19 2050",
 "id_pre_26":"EI 3.8 cutoff image SSP2-RCP26 2050",
"id_re_base":"EI 3.8 cutoff remind SSP2-Base 2050",
"id_re_1150":"EI 3.8 cutoff remind SSP2-PkBudg1150 2050",
"id_re_1150":"EI 3.8 cutoff remind SSP2-PkBudg500 2050"
}
for idx, row in df.iterrows():
    act_name = row["name"]
    act_location = row["location"]
    act_ref = row["reference product"]
    row_for_df = {"name":row["name"]}
    
    for name_short, name_long in sce_names_dict.items():
        try:
            act_bg_id = bd.get_activity(database = name_long,name=act_name,location=act_location, product = act_ref).id
        except:
            act_bg_id =row["id"]
        row_for_df[name_short] = act_bg_id

        
    id_all.append(row_for_df)
scenarios_ids=pd.DataFrame(id_all)    
scenarios_ids = scenarios_ids.set_index("id_ei38", drop=False)
scenarios_ids

,name,id_ei38,id_pre_base,id_pre_19,id_pre_26,id_re_base,id_re_1150
id_ei38,,,,,,,
6599,"electricity production, hydro, run-of-river",6599,92881,140945,116913,164977,209765
8302,"operation, solar collector system, Cu flat pla...",8302,89605,137669,113637,161701,206489
10319,"electricity production, wind, 1-3MW turbine, o...",10319,91387,139451,115419,163483,208271
18352,"electricity production, photovoltaic, 3kWp sla...",18352,92843,140907,116875,164939,209727
20079,"heat and power co-generation, wood chips, 6667...",20079,89007,137071,113039,161103,205891
23758,"heat production, wood pellet, at furnace 25kW,...",23758,87162,135226,111194,159258,204046
213909,"electricity supply, Hotel Moeschberg, 2021",213909,213909,213909,213909,213909,213909
213910,"electricity supply, Energie Grosshoechstetten ...",213910,213910,213910,213910,213910,213910
213911,"electricity, subsidised, 2021",213911,213911,213911,213911,213911,213911


In [65]:
import pandas as pd
matrix_raw = pd.DataFrame({"row":rows, 
                           "col":cols, 
                           "from":[mapping_act[idx] for idx in rows], 
                           "to":[mapping_act[idx] for idx in cols],
                           "data":data
                          })
matrix = matrix_raw.pivot(index='row',columns='col', values='data').fillna(0)
# mapping
# matrix_raw

In [66]:
# objects[2].metadata["name"]
# dp_dict = dict(
#         zip(indices.astype("object"),[[tup[0], tup[1]] for tup in zip(data,flip)])
#                 )
# dp_dict

{(6599, 213910): [0.907, True],
 (6599, 213911): [0.32, True],
 (8302, 213912): [0.004668, True],
 (10319, 213911): [0.02, True],
 (18352, 213909): [0.4406, True],
 (18352, 213910): [0.026, True],
 (18352, 213911): [0.46, True],
 (20079, 213911): [0.2, True],
 (23758, 213912): [0.9953, True],
 (213909, 213909): [1.0, False],
 (213909, 213913): [32179.0, True],
 (213910, 213909): [0.5594, True],
 (213910, 213910): [1.0, False],
 (213911, 213910): [0.067, True],
 (213911, 213911): [1.0, False],
 (213912, 213912): [1.0, False],
 (213912, 213913): [655489.0, True],
 (213913, 213913): [1.0, False]}

In [22]:
new_data = np.array([val[0] for val in dp_dict.items()])

In [23]:
mapping_act

{6599: 'electricity production, hydro, run-of-river',
 8302: 'operation, solar collector system, Cu flat plate collector, one-family house, for hot water',
 10319: 'electricity production, wind, 1-3MW turbine, onshore',
 18352: 'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted',
 20079: 'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014',
 23758: 'heat production, wood pellet, at furnace 25kW, state-of-the-art 2014',
 213909: 'electricity supply, Hotel Moeschberg, 2021',
 213910: 'electricity supply, Energie Grosshoechstetten AG, 2021, guarantees of origin',
 213911: 'electricity, subsidised, 2021',
 213912: 'heat supply, Hotel Moeschberg, 2021',
 213913: 'energy demand, operational, Hotel Moeschberg'}

In [95]:
bg_base = "EI 3.8 cutoff image SSP2-Base 2050"
bg_19 = "EI 3.8 cutoff image SSP2-RCP19 2050"
bgs = [bg_base, bg_19]
db=bd.Database(bg_base)

In [91]:

def modify_w_arrays(data_object, new_arrays, new_indices_dict):
    n = 1
    for k,v in new_arrays.items():
        n = len(v)        

    data, _ = data_object.get_resource(f"{data_object.metadata['name']}_technosphere_matrix.data")
    indices, _ = data_object.get_resource(f"{data_object.metadata['name']}_technosphere_matrix.indices")
    flip, _ = data_object.get_resource(f"{data_object.metadata['name']}_technosphere_matrix.flip")
    
    
    dp_dict = dict(
        zip(indices.astype("object"),[[tup[0], tup[1]] for tup in zip(data,flip)])
                        )
    for key, value in new_arrays.items():
        dp_dict[key][0] = value
    
    new_foreground = bwp.create_datapackage(
    fs = bwp.generic_zipfile_filesystem(dirpath=Path("./data/inputs"), filename=f"{data_object.metadata['name']}.zip", write=True),
    # combinatorial=True,
    sequential=True,
    )

    for row_col, data_flip in dp_dict.items():
        if not isinstance(data_flip[0], np.ndarray):
            new_array = np.full((n,), fill_value = data_flip[0])
            dp_dict[row_col][0] = new_array
            
    new_data = np.array([val[0] for val in dp_dict.items()])
    
    new_indices=[]
    for row, col in indices:
        new_indices.append((new_indices_dict[row], col))
               
        
    new_foreground.add_persistent_array(
    matrix="technosphere_matrix",
    data_array=new_data,
    indices_array=np.array(new_indices, dtype=bwp.INDICES_DTYPE),  
    flip_array=flip,
    name=data_object.metadata['name'],
)

    # return dp_dict
    return new_foreground

In [92]:
new_ind_dict

{6599: 6599,
 8302: 8302,
 10319: 10319,
 18352: 18352,
 20079: 20079,
 23758: 23758,
 213909: 213909,
 213910: 213910,
 213911: 213911,
 213912: 213912,
 213913: 213913}

In [103]:
# bg_base = "EI 3.8 cutoff image SSP2-Base 2050"
# bg_19 = "EI 3.8 cutoff image SSP2-RCP19 2050"
# bgs = [bg_base, bg_19]



fu = {bd.get_activity(database="energy_moeschberg", name = "energy demand, operational, Hotel Moeschberg"):1}
fu, objects, remapping_dicts = bd.prepare_lca_inputs(fu, method=('IPCC 2013', 'climate change', 'GWP 100a'))

for col in scenarios_ids.iteritems():
    scenario_name = col[0]
    if "name" not in scenario_name: 
        print("###################################################################################################")
        print("SCENARIO CALCULATION: ", scenario_name)
        print("###################################################################################################")
        new_ind_dict= {}
        for idx, row in scenarios_ids.iterrows():
            original_id = idx
            new_id = row[scenario_name]
            new_ind_dict[original_id]=new_id
            
            new_arrays = {
                        (213912,213913): [655489,655489*0.5], #PV becomes the unique elec source
                        (213909,213913): [32179,32179*1.5],
            }
        new_foreground = modify_w_arrays(objects[2], new_arrays, new_ind_dict)
        db=bd.Database(sce_names_dict[scenario_name])
        db_dp = db.datapackage()
        
        
        lca_b = bc.LCA(
        demand=fu,
        data_objs=objects + [db_dp] + [new_foreground],
        use_distributions=False,
        use_arrays=True,
        )
        lca_b.lci()
        lca_b.lcia()
        lca_b.keep_first_iteration()
        iterations = 4
        scores_b = [lca_b.score for _ in zip(range(iterations), lca_b)]
        print("RESULTS", scores_b)


/tmp/ipykernel_765224/2229086869.py:10: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col in scenarios_ids.iteritems():


###################################################################################################
SCENARIO CALCULATION:  id_ei38
###################################################################################################
RESULTS [0.027706061564151483, 0.33116044116494214, 0.027706061564151483, 0.33116044116494214]
###################################################################################################
SCENARIO CALCULATION:  id_pre_base
###################################################################################################
RESULTS [0.03214454703542381, 0.370111332696073, 0.03214454703542381, 0.370111332696073]
###################################################################################################
SCENARIO CALCULATION:  id_pre_19
###################################################################################################
RESULTS [0.02375386848858267, 0.2730573120844077, 0.02375386848858267, 0.2730573120844077]
##########################

In [32]:
aa = modify_w_arrays(objects[2], new_arrays)

In [62]:
for idx, col in scenarios_ids.iteritems():
    print(col)

0           electricity production, hydro, run-of-river
1     operation, solar collector system, Cu flat pla...
2     electricity production, wind, 1-3MW turbine, o...
3     electricity production, photovoltaic, 3kWp sla...
4     heat and power co-generation, wood chips, 6667...
5     heat production, wood pellet, at furnace 25kW,...
6            electricity supply, Hotel Moeschberg, 2021
7     electricity supply, Energie Grosshoechstetten ...
8                         electricity, subsidised, 2021
9                   heat supply, Hotel Moeschberg, 2021
10         energy demand, operational, Hotel Moeschberg
Name: name, dtype: object
0       6599
1       8302
2      10319
3      18352
4      20079
5      23758
6     213909
7     213910
8     213911
9     213912
10    213913
Name: id_ei38, dtype: int32
0      92881
1      89605
2      91387
3      92843
4      89007
5      87162
6     213909
7     213910
8     213911
9     213912
10    213913
Name: id_pre_base, dtype: int64
0     140945

/tmp/ipykernel_747396/1499316733.py:1: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for idx, col in scenarios_ids.iteritems():


In [33]:
for idx, col in scenarios_ids.iteritems()

    lca_b = bc.LCA(
        demand=fu,
        data_objs=objects + [aa],
        use_distributions=False,
        use_arrays=True,
    #     seed_override=42,  # Seed should not be used
    )
    lca_b.lci()
    lca_b.lcia()
    lca_b.keep_first_iteration()
    iterations = 2
    scores_b = [lca_b.score for _ in zip(range(iterations), lca_b)]
    print("SCENARIO CALC", scores_b)

[0.027706061564151483,
 0.33116044116494214,
 0.027706061564151483,
 0.33116044116494214,
 0.027706061564151483,
 0.33116044116494214,
 0.027706061564151483,
 0.33116044116494214,
 0.027706061564151483,
 0.33116044116494214]